# White-box Attack on TinyImagenet

In [1]:
import sys
#
import torch
import torch.nn as nn

#sys.path.insert(0, '..')
# import torchattacks

In [2]:
%cd adversarial-attacks-pytorch
import torchattacks
import robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
%cd ..

/Users/alpayozkan/opt/anaconda3/envs/ex4/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/alpayozkan/eth/dl_proj/adversarial-attacks-pytorch
/Users/alpayozkan/eth/dl_proj


/Users/alpayozkan/opt/anaconda3/envs/ex4/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [110]:
# imports
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn import CrossEntropyLoss
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from utils.metrics import topk_acc, real_acc, AverageMeter
from utils.parsers import get_training_parser
from utils.optimizer import get_optimizer, get_scheduler, OPTIMIZERS_DICT, SCHEDULERS

from models.networks import get_model, get_model_bypass
from data_utils.data_stats import *

import matplotlib.pyplot as plt
import argparse
import time

In [4]:
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

torch.manual_seed(0)

In [5]:
from torchattacks import PGD
from utils_attack import *

In [6]:
device = 'cuda'
print(torch.cuda.is_available())

False


In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Assuming a single-GPU setup
    device_properties = torch.cuda.get_device_properties(device)
    
    total_memory = device_properties.total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    cached_memory = torch.cuda.memory_reserved(device)

    print(f"Total GPU Memory: {total_memory / 1e9} GB")
    print(f"Allocated Memory: {allocated_memory / 1e9} GB")
    print(f"Cached Memory: {cached_memory / 1e9} GB")
else:
    print("CUDA is not available.")

CUDA is not available.


# LOAD MLP

In [48]:
dataset = 'tinyimagenet'                 # One of cifar10, cifar100, stl10, imagenet or imagenet21
architecture = 'B_12-Wi_1024'
data_resolution = 64                # Resolution of data as it is stored
crop_resolution = 64                # Resolution of fine-tuned model (64 for all models we provide)
num_classes = CLASS_DICT[dataset]
eval_batch_size = 1024
checkpoint = 'B_12-Wi_1024_res_64_in21k_tinyimagenet'

In [38]:
torch.backends.cuda.matmul.allow_tf32 = True

In [39]:
num_classes

200

In [71]:
import json

from torch import nn
import torch
import numpy as np

from utils.download import download, default_checkpoints


NORMS = {
    'layer': nn.LayerNorm,
    'batch': nn.BatchNorm1d,
    'none': nn.Identity
}

ACT = {
    'gelu': nn.GELU(),
    'relu': nn.ReLU()
}


class StandardMLP(nn.Module):
    def __init__(self, dim_in, dim_out, widths):
        super(StandardMLP, self).__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.widths = widths
        self.linear_in = nn.Linear(self.dim_in, self.widths[0])
        self.linear_out = nn.Linear(self.widths[-1], self.dim_out)
        self.layers = []
        self.layer_norms = []
        for i in range(len(self.widths) - 1):
            self.layers.append(nn.Linear(self.widths[i], self.widths[i + 1]))
            self.layer_norms.append(nn.LayerNorm(widths[i + 1]))

        self.layers = nn.ModuleList(self.layers)
        self.layernorms = nn.ModuleList(self.layer_norms)

    def forward(self, x):
        z = self.linear_in(x)
        for layer, norm in zip(self.layers, self.layer_norms):
            z = norm(z)
            z = nn.GELU()(z)
            z = layer(z)

        out = self.linear_out(z)

        return out


class BottleneckMLP(nn.Module):
    def __init__(self, dim_in, dim_out, block_dims, norm='layer', checkpoint=None, name=None, bypass=False):
        super(BottleneckMLP, self).__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.block_dims = block_dims
        self.norm = NORMS[norm]
        self.checkpoint = checkpoint

        self.name = name
        self.linear_in = nn.Linear(self.dim_in, self.block_dims[0][1])
        self.linear_out = nn.Linear(self.block_dims[-1][1], self.dim_out)
        blocks = []
        layernorms = []

        for block_dim in self.block_dims:
            wide, thin = block_dim
            blocks.append(BottleneckBlock(thin=thin, wide=wide))
            layernorms.append(self.norm(thin))

        self.blocks = nn.ModuleList(blocks)
        self.layernorms = nn.ModuleList(layernorms)

        if self.checkpoint is not None:
            if not bypass:
                self.load(self.checkpoint)
            else:
                self.load_bypass(self.checkpoint)

    def forward(self, x):
        x = self.linear_in(x)

        for block, norm in zip(self.blocks, self.layernorms):
            x = x + block(norm(x))

        out = self.linear_out(x)

        return out

    def load(self, name, checkpoint_path='./checkpoints/'):
        #if name == True:
            # This simply assumes Imagenet21 pre-trained weights at the latest epoch available, no fine-tuning
        #    name = default_checkpoints[self.name]
        #elif name in ['cifar10', 'cifar100', 'imagenet']:
            # This loads the optimal fine-tuned weights for that dataset
        #    name = default_checkpoints[self.name + '_' + name]
        #else:
            # This assumes a full path, e.g. also specifying which epoch etc
        #    name = self.name + '_' + name
        name = self.name + '_' + name
        name = default_checkpoints[name]
        weight_path, config_path = download(name, checkpoint_path)

        with open(config_path, 'r') as f:
            self.config = json.load(f)

        params = {
            k: v
            for k, v in torch.load(weight_path, map_location ='cpu').items()
        }

        # Load pre-trained parameters
        print('Load_state output', self.load_state_dict(params, strict=False))

    def load_bypass(self, name, checkpoint_path='./checkpoints/'):
        #if name == True:
            # This simply assumes Imagenet21 pre-trained weights at the latest epoch available, no fine-tuning
        #    name = default_checkpoints[self.name]
        #elif name in ['cifar10', 'cifar100', 'imagenet']:
            # This loads the optimal fine-tuned weights for that dataset
        #    name = default_checkpoints[self.name + '_' + name]
        #else:
            # This assumes a full path, e.g. also specifying which epoch etc
        #    name = self.name + '_' + name
        weight_path = checkpoint_path + name
        
        print("bypass")

        params = {
            k: v
            for k, v in torch.load(weight_path, map_location ='cpu').items()
        }

        # Load pre-trained parameters
        print('Load_state output', self.load_state_dict(params, strict=True))


class BottleneckBlock(nn.Module):
    def __init__(self, thin, wide, act=nn.GELU()):
        super(BottleneckBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(thin, wide), act, nn.Linear(wide, thin)
        )

    def forward(self, x):
        out = self.block(x)

        return out


def B_12_Wi_1024(dim_in, dim_out, checkpoint=None, bypass=False):
    block_dims = [[4 * 1024, 1024] for _ in range(12)]
    return BottleneckMLP(dim_in=dim_in, dim_out=dim_out, norm='layer', block_dims=block_dims, checkpoint=checkpoint,
                         name='B_' + str(len(block_dims)) + '-Wi_' + str(block_dims[0][1]) + '_res_' + str(int(np.sqrt(dim_in/3))), bypass=bypass)


def B_12_Wi_512(dim_in, dim_out, checkpoint=None, bypass=False):
    block_dims = [[4 * 512, 512] for _ in range(12)]
    return BottleneckMLP(dim_in=dim_in, dim_out=dim_out, norm='layer', block_dims=block_dims, checkpoint=checkpoint,
                         name='B_' + str(len(block_dims)) + '-Wi_' + str(block_dims[0][1]) + '_res_' + str(int(np.sqrt(dim_in/3))), bypass=bypass)


def B_6_Wi_1024(dim_in, dim_out, checkpoint=None, bypass=False):
    block_dims = [[4 * 1024, 1024] for _ in range(6)]
    return BottleneckMLP(dim_in=dim_in, dim_out=dim_out, norm='layer', block_dims=block_dims, checkpoint=checkpoint,
                         name='B_' + str(len(block_dims)) + '-Wi_' + str(block_dims[0][1]) + '_res_' + str(int(np.sqrt(dim_in/3))), bypass=bypass)


def B_6_Wi_512(dim_in, dim_out, checkpoint=None, bypass=False):
    block_dims = [[4 * 512, 512] for _ in range(6)]
    return BottleneckMLP(dim_in=dim_in, dim_out=dim_out, norm='layer', block_dims=block_dims, checkpoint=checkpoint,
                         name='B_' + str(len(block_dims)) + '-Wi_' + str(block_dims[0][1]) + '_res_' + str(int(np.sqrt(dim_in/3))), bypass=bypass)


model_list = {
    'B_12-Wi_1024': B_12_Wi_1024,
    'B_12-Wi_512': B_12_Wi_512,
    'B_6-Wi_1024': B_6_Wi_1024,
    'B_6-Wi_512': B_6_Wi_512
}


def get_model(architecture, checkpoint, resolution, num_classes):
    return model_list[architecture](dim_in=resolution**2 * 3, dim_out=num_classes, checkpoint=checkpoint)

def get_model_bypass(architecture, checkpoint, resolution, num_classes):
    return model_list[architecture](dim_in=resolution**2 * 3, dim_out=num_classes, checkpoint=checkpoint, bypass=True)

In [72]:
class MLP_Wrapper(nn.Module):
    def __init__(self, architecture, resolution, num_classes, checkpoint):
        super(MLP_Wrapper, self).__init__()
        self.model = get_model_bypass(architecture=architecture, resolution=crop_resolution, num_classes=num_classes, checkpoint=checkpoint)
        self.resize = transforms.Resize((resolution, resolution))

    def forward(self, x):
        x = self.resize(x)
        x = x.flatten(1)
        x = self.model(x)
        return x

In [73]:
dd1.keys()

odict_keys(['module.linear_in.weight', 'module.linear_in.bias', 'module.linear_out.weight', 'module.linear_out.bias', 'module.blocks.0.block.0.weight', 'module.blocks.0.block.0.bias', 'module.blocks.0.block.2.weight', 'module.blocks.0.block.2.bias', 'module.blocks.1.block.0.weight', 'module.blocks.1.block.0.bias', 'module.blocks.1.block.2.weight', 'module.blocks.1.block.2.bias', 'module.blocks.2.block.0.weight', 'module.blocks.2.block.0.bias', 'module.blocks.2.block.2.weight', 'module.blocks.2.block.2.bias', 'module.blocks.3.block.0.weight', 'module.blocks.3.block.0.bias', 'module.blocks.3.block.2.weight', 'module.blocks.3.block.2.bias', 'module.blocks.4.block.0.weight', 'module.blocks.4.block.0.bias', 'module.blocks.4.block.2.weight', 'module.blocks.4.block.2.bias', 'module.blocks.5.block.0.weight', 'module.blocks.5.block.0.bias', 'module.blocks.5.block.2.weight', 'module.blocks.5.block.2.bias', 'module.blocks.6.block.0.weight', 'module.blocks.6.block.0.bias', 'module.blocks.6.block.2

In [74]:
model_mlp = MLP_Wrapper(architecture, crop_resolution, num_classes, checkpoint)

model_mlp = model_mlp.to(device)
model_mlp.eval()

bypass


RuntimeError: Error(s) in loading state_dict for BottleneckMLP:
	Missing key(s) in state_dict: "linear_in.weight", "linear_in.bias", "linear_out.weight", "linear_out.bias", "blocks.0.block.0.weight", "blocks.0.block.0.bias", "blocks.0.block.2.weight", "blocks.0.block.2.bias", "blocks.1.block.0.weight", "blocks.1.block.0.bias", "blocks.1.block.2.weight", "blocks.1.block.2.bias", "blocks.2.block.0.weight", "blocks.2.block.0.bias", "blocks.2.block.2.weight", "blocks.2.block.2.bias", "blocks.3.block.0.weight", "blocks.3.block.0.bias", "blocks.3.block.2.weight", "blocks.3.block.2.bias", "blocks.4.block.0.weight", "blocks.4.block.0.bias", "blocks.4.block.2.weight", "blocks.4.block.2.bias", "blocks.5.block.0.weight", "blocks.5.block.0.bias", "blocks.5.block.2.weight", "blocks.5.block.2.bias", "blocks.6.block.0.weight", "blocks.6.block.0.bias", "blocks.6.block.2.weight", "blocks.6.block.2.bias", "blocks.7.block.0.weight", "blocks.7.block.0.bias", "blocks.7.block.2.weight", "blocks.7.block.2.bi

In [12]:
num_classes

200

In [13]:
checkpoint

'B_12-Wi_1024_res_64_in21k_tinyimagenet.t7'

In [14]:
checkpoint

'B_12-Wi_1024_res_64_in21k_tinyimagenet.t7'

In [18]:
device = 'cpu'

In [22]:
architecture

'B_12-Wi_1024'

In [44]:
dd['model']

OrderedDict([('module.linear_in.weight',
              tensor([[ 2.2785e-01,  2.9513e-01,  3.1012e-03,  ..., -1.0594e-01,
                        2.2147e-02,  8.5203e-02],
                      [-2.3599e-01, -5.2422e-02, -4.1217e-02,  ..., -1.1177e-01,
                       -4.4740e-01, -4.3109e-01],
                      [-4.5495e-01, -4.4787e-01, -2.9203e-01,  ...,  1.4033e-01,
                        2.5422e-01,  3.6087e-01],
                      ...,
                      [ 1.7607e-02, -3.5418e-01,  1.9371e-02,  ..., -2.6691e-01,
                       -5.2200e-01, -2.3111e-01],
                      [ 3.8978e-01,  1.9745e-01,  4.5564e-01,  ...,  2.0694e-01,
                        4.6923e-01,  3.8922e-01],
                      [-1.0741e-02,  2.3873e-05,  6.7596e-02,  ..., -3.2074e-02,
                       -1.3536e-02, -9.1601e-02]])),
             ('module.linear_in.bias',
              tensor([ 1.0549,  0.4441, -0.6998,  ...,  0.7105,  1.1405, -0.3989])),
             ('modu

In [28]:
dd = torch.load('checkpoints/B_12-Wi_1024_res_64_in21k_tinyimagenet.t7', map_location='cpu')

In [45]:
torch.save(dd['model'], 'checkpoints/B_12-Wi_1024_res_64_in21k_tinyimagenet')

In [54]:
dd1 = torch.load('checkpoints/B_12-Wi_1024_res_64_in21k_tinyimagenet', map_location='cpu')

In [77]:
dd1.keys()

odict_keys(['module.linear_in.weight', 'module.linear_in.bias', 'module.linear_out.weight', 'module.linear_out.bias', 'module.blocks.0.block.0.weight', 'module.blocks.0.block.0.bias', 'module.blocks.0.block.2.weight', 'module.blocks.0.block.2.bias', 'module.blocks.1.block.0.weight', 'module.blocks.1.block.0.bias', 'module.blocks.1.block.2.weight', 'module.blocks.1.block.2.bias', 'module.blocks.2.block.0.weight', 'module.blocks.2.block.0.bias', 'module.blocks.2.block.2.weight', 'module.blocks.2.block.2.bias', 'module.blocks.3.block.0.weight', 'module.blocks.3.block.0.bias', 'module.blocks.3.block.2.weight', 'module.blocks.3.block.2.bias', 'module.blocks.4.block.0.weight', 'module.blocks.4.block.0.bias', 'module.blocks.4.block.2.weight', 'module.blocks.4.block.2.bias', 'module.blocks.5.block.0.weight', 'module.blocks.5.block.0.bias', 'module.blocks.5.block.2.weight', 'module.blocks.5.block.2.bias', 'module.blocks.6.block.0.weight', 'module.blocks.6.block.0.bias', 'module.blocks.6.block.2

In [87]:
c=0
c2=0
for k in dd1:
    if k[:7] != 'module.':
        c += 1
    else:
        c2 +=1

In [ ]:

for k in dd1:


In [85]:
k[7:]

'linear_in.weight'

In [97]:
dd_cpy = dd1.copy()

In [98]:
for k in dd1:
    val = dd1[k]
    dd_cpy.pop(k)
    dd_cpy[k[7:]] = val

In [99]:
dd_cpy

OrderedDict([('linear_in.weight',
              tensor([[ 2.2785e-01,  2.9513e-01,  3.1012e-03,  ..., -1.0594e-01,
                        2.2147e-02,  8.5203e-02],
                      [-2.3599e-01, -5.2422e-02, -4.1217e-02,  ..., -1.1177e-01,
                       -4.4740e-01, -4.3109e-01],
                      [-4.5495e-01, -4.4787e-01, -2.9203e-01,  ...,  1.4033e-01,
                        2.5422e-01,  3.6087e-01],
                      ...,
                      [ 1.7607e-02, -3.5418e-01,  1.9371e-02,  ..., -2.6691e-01,
                       -5.2200e-01, -2.3111e-01],
                      [ 3.8978e-01,  1.9745e-01,  4.5564e-01,  ...,  2.0694e-01,
                        4.6923e-01,  3.8922e-01],
                      [-1.0741e-02,  2.3873e-05,  6.7596e-02,  ..., -3.2074e-02,
                       -1.3536e-02, -9.1601e-02]])),
             ('linear_in.bias',
              tensor([ 1.0549,  0.4441, -0.6998,  ...,  0.7105,  1.1405, -0.3989])),
             ('linear_out.weight'

In [100]:
torch.save(dd_cpy, 'checkpoints/B_12-Wi_1024_res_64_in21k_tinyimagenet')

In [84]:
dd1.keys()

odict_keys(['module.linear_in.weight', 'module.linear_in.bias', 'module.linear_out.weight', 'module.linear_out.bias', 'module.blocks.0.block.0.weight', 'module.blocks.0.block.0.bias', 'module.blocks.0.block.2.weight', 'module.blocks.0.block.2.bias', 'module.blocks.1.block.0.weight', 'module.blocks.1.block.0.bias', 'module.blocks.1.block.2.weight', 'module.blocks.1.block.2.bias', 'module.blocks.2.block.0.weight', 'module.blocks.2.block.0.bias', 'module.blocks.2.block.2.weight', 'module.blocks.2.block.2.bias', 'module.blocks.3.block.0.weight', 'module.blocks.3.block.0.bias', 'module.blocks.3.block.2.weight', 'module.blocks.3.block.2.bias', 'module.blocks.4.block.0.weight', 'module.blocks.4.block.0.bias', 'module.blocks.4.block.2.weight', 'module.blocks.4.block.2.bias', 'module.blocks.5.block.0.weight', 'module.blocks.5.block.0.bias', 'module.blocks.5.block.2.weight', 'module.blocks.5.block.2.bias', 'module.blocks.6.block.0.weight', 'module.blocks.6.block.0.bias', 'module.blocks.6.block.2

In [61]:
params = {
    k: v
    for k, v in dd1.items()
}

In [62]:
params

{'module.linear_in.weight': tensor([[ 2.2785e-01,  2.9513e-01,  3.1012e-03,  ..., -1.0594e-01,
           2.2147e-02,  8.5203e-02],
         [-2.3599e-01, -5.2422e-02, -4.1217e-02,  ..., -1.1177e-01,
          -4.4740e-01, -4.3109e-01],
         [-4.5495e-01, -4.4787e-01, -2.9203e-01,  ...,  1.4033e-01,
           2.5422e-01,  3.6087e-01],
         ...,
         [ 1.7607e-02, -3.5418e-01,  1.9371e-02,  ..., -2.6691e-01,
          -5.2200e-01, -2.3111e-01],
         [ 3.8978e-01,  1.9745e-01,  4.5564e-01,  ...,  2.0694e-01,
           4.6923e-01,  3.8922e-01],
         [-1.0741e-02,  2.3873e-05,  6.7596e-02,  ..., -3.2074e-02,
          -1.3536e-02, -9.1601e-02]]),
 'module.linear_in.bias': tensor([ 1.0549,  0.4441, -0.6998,  ...,  0.7105,  1.1405, -0.3989]),
 'module.linear_out.weight': tensor([[ 1.5310e-02,  4.5584e-03, -2.1657e-02,  ...,  1.4852e-02,
           1.7602e-02,  3.7569e-02],
         [-2.4131e-02,  4.3384e-02,  2.5383e-04,  ..., -1.0433e-02,
           6.3600e-02,  8.5815

In [ ]:
print('Load_state output', self.load_state_dict(params, strict=False))

In [59]:
model_mlp.parameters

<bound method Module.parameters of MLP_Wrapper(
  (model): BottleneckMLP(
    (linear_in): Linear(in_features=12288, out_features=1024, bias=True)
    (linear_out): Linear(in_features=1024, out_features=200, bias=True)
    (blocks): ModuleList(
      (0-11): 12 x BottleneckBlock(
        (block): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (layernorms): ModuleList(
      (0-11): 12 x LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
  )
  (resize): Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
)>

In [32]:
dd.keys()

dict_keys(['model', 'optimizer', 'scaler', 'epoch', 'best_acc'])

In [33]:
dd2 = torch.load('checkpoints/B-12_Wi-1024_res_64_cifar10_epochs_20_weights', map_location='cpu')

In [34]:
dd2

OrderedDict([('linear_in.weight',
              tensor([[ 2.2766e-01,  2.9493e-01,  2.6623e-03,  ..., -1.0544e-01,
                        2.2706e-02,  8.5756e-02],
                      [-2.3604e-01, -5.2510e-02, -4.1368e-02,  ..., -1.1234e-01,
                       -4.4782e-01, -4.3150e-01],
                      [-4.5428e-01, -4.4719e-01, -2.9147e-01,  ...,  1.3970e-01,
                        2.5342e-01,  3.6002e-01],
                      ...,
                      [ 1.5980e-02, -3.5570e-01,  1.7743e-02,  ..., -2.6705e-01,
                       -5.2203e-01, -2.3121e-01],
                      [ 3.9012e-01,  1.9785e-01,  4.5613e-01,  ...,  2.0843e-01,
                        4.7058e-01,  3.9061e-01],
                      [-1.0476e-02,  2.8890e-04,  6.7830e-02,  ..., -3.1759e-02,
                       -1.3554e-02, -9.1598e-02]])),
             ('linear_in.bias',
              tensor([ 1.0531,  0.4436, -0.6995,  ...,  0.7111,  1.1395, -0.3991])),
             ('linear_out.weight'

In [75]:
dd2.keys()

odict_keys(['linear_in.weight', 'linear_in.bias', 'linear_out.weight', 'linear_out.bias', 'blocks.0.block.0.weight', 'blocks.0.block.0.bias', 'blocks.0.block.2.weight', 'blocks.0.block.2.bias', 'blocks.1.block.0.weight', 'blocks.1.block.0.bias', 'blocks.1.block.2.weight', 'blocks.1.block.2.bias', 'blocks.2.block.0.weight', 'blocks.2.block.0.bias', 'blocks.2.block.2.weight', 'blocks.2.block.2.bias', 'blocks.3.block.0.weight', 'blocks.3.block.0.bias', 'blocks.3.block.2.weight', 'blocks.3.block.2.bias', 'blocks.4.block.0.weight', 'blocks.4.block.0.bias', 'blocks.4.block.2.weight', 'blocks.4.block.2.bias', 'blocks.5.block.0.weight', 'blocks.5.block.0.bias', 'blocks.5.block.2.weight', 'blocks.5.block.2.bias', 'blocks.6.block.0.weight', 'blocks.6.block.0.bias', 'blocks.6.block.2.weight', 'blocks.6.block.2.bias', 'blocks.7.block.0.weight', 'blocks.7.block.0.bias', 'blocks.7.block.2.weight', 'blocks.7.block.2.bias', 'blocks.8.block.0.weight', 'blocks.8.block.0.bias', 'blocks.8.block.2.weight',

In [76]:
dd1.keys()

odict_keys(['module.linear_in.weight', 'module.linear_in.bias', 'module.linear_out.weight', 'module.linear_out.bias', 'module.blocks.0.block.0.weight', 'module.blocks.0.block.0.bias', 'module.blocks.0.block.2.weight', 'module.blocks.0.block.2.bias', 'module.blocks.1.block.0.weight', 'module.blocks.1.block.0.bias', 'module.blocks.1.block.2.weight', 'module.blocks.1.block.2.bias', 'module.blocks.2.block.0.weight', 'module.blocks.2.block.0.bias', 'module.blocks.2.block.2.weight', 'module.blocks.2.block.2.bias', 'module.blocks.3.block.0.weight', 'module.blocks.3.block.0.bias', 'module.blocks.3.block.2.weight', 'module.blocks.3.block.2.bias', 'module.blocks.4.block.0.weight', 'module.blocks.4.block.0.bias', 'module.blocks.4.block.2.weight', 'module.blocks.4.block.2.bias', 'module.blocks.5.block.0.weight', 'module.blocks.5.block.0.bias', 'module.blocks.5.block.2.weight', 'module.blocks.5.block.2.bias', 'module.blocks.6.block.0.weight', 'module.blocks.6.block.0.bias', 'module.blocks.6.block.2

In [24]:
model_mlp

MLP_Wrapper(
  (model): BottleneckMLP(
    (linear_in): Linear(in_features=12288, out_features=1024, bias=True)
    (linear_out): Linear(in_features=1024, out_features=200, bias=True)
    (blocks): ModuleList(
      (0-11): 12 x BottleneckBlock(
        (block): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (layernorms): ModuleList(
      (0-11): 12 x LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
  )
  (resize): Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
)

In [50]:
architecture

'B_12-Wi_1024'

In [51]:
checkpoint

'B_12-Wi_1024_res_64_in21k_tinyimagenet'

Load_state output _IncompatibleKeys(missing_keys=['linear_in.weight', 'linear_in.bias', 'linear_out.weight', 'linear_out.bias', 'blocks.0.block.0.weight', 'blocks.0.block.0.bias', 'blocks.0.block.2.weight', 'blocks.0.block.2.bias', 'blocks.1.block.0.weight', 'blocks.1.block.0.bias', 'blocks.1.block.2.weight', 'blocks.1.block.2.bias', 'blocks.2.block.0.weight', 'blocks.2.block.0.bias', 'blocks.2.block.2.weight', 'blocks.2.block.2.bias', 'blocks.3.block.0.weight', 'blocks.3.block.0.bias', 'blocks.3.block.2.weight', 'blocks.3.block.2.bias', 'blocks.4.block.0.weight', 'blocks.4.block.0.bias', 'blocks.4.block.2.weight', 'blocks.4.block.2.bias', 'blocks.5.block.0.weight', 'blocks.5.block.0.bias', 'blocks.5.block.2.weight', 'blocks.5.block.2.bias', 'blocks.6.block.0.weight', 'blocks.6.block.0.bias', 'blocks.6.block.2.weight', 'blocks.6.block.2.bias', 'blocks.7.block.0.weight', 'blocks.7.block.0.bias', 'blocks.7.block.2.weight', 'blocks.7.block.2.bias', 'blocks.8.block.0.weight', 'blocks.8.blo

In [101]:
model_mlp = MLP_Wrapper(architecture, crop_resolution, num_classes, checkpoint)

model_mlp = model_mlp.to(device)
model_mlp.eval()

bypass
Load_state output <All keys matched successfully>


MLP_Wrapper(
  (model): BottleneckMLP(
    (linear_in): Linear(in_features=12288, out_features=1024, bias=True)
    (linear_out): Linear(in_features=1024, out_features=200, bias=True)
    (blocks): ModuleList(
      (0-11): 12 x BottleneckBlock(
        (block): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (layernorms): ModuleList(
      (0-11): 12 x LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
  )
  (resize): Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
)

# LOAD CNN

In [103]:
from cifar10_models.resnet import *
model_resnet = resnet18()

# Use the weights of the resnet18 model trained on CIFAR10
weights = torch.load('state_dicts/resnet18.pt')
model_resnet.load_state_dict(weights)

model_resnet = model_resnet.to(device)
model_resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [104]:
print(model_resnet.fc.weight.is_cuda)

False


In [ ]:
asads

# LOAD ViT

In [ ]:
import timm
model_ViT_tmp = timm.create_model("vit_base_patch16_384", pretrained=True)
model_ViT_tmp.head = nn.Linear(model_ViT_tmp.head.in_features, 10) # CIFAR10
checkpoint = torch.load("finetuned_models/ViT_CIFAR10.t7", map_location=torch.device("cpu"))
model_ViT_tmp = torch.nn.DataParallel(model_ViT_tmp)
model_ViT_tmp.load_state_dict(checkpoint['model'])
print("model loaded")
# ViT uses 384x384 pixel image
input_size = 384

In [ ]:
class ViT_Wrapper(nn.Module):
    def __init__(self, model, input_size):
        super(ViT_Wrapper, self).__init__()
        self.model = model
        self.resize = transforms.Resize(input_size)

    def forward(self, x):
        x = self.resize(x)
        x = self.model(x)
        return x

In [ ]:
model_ViT = ViT_Wrapper(model_ViT_tmp, input_size)
model_ViT.eval()

# LOAD DATA

In [128]:
from datasets import load_dataset, get_dataset_split_names
import cv2

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
input_size = 64
bs = 100

transform_test = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

#batch_size = 100

from PIL import Image


def collate_fn_test(batch):
    return ((torch.stack([transform_test(Image.fromarray(cv2.cvtColor(np.array(x['image']),cv2.COLOR_GRAY2RGB))) if np.array(x['image']).ndim == 2 else transform_test(x['image']) for x in batch])), torch.tensor([x['label'] for x in batch]))
    
testset = load_dataset('Maysee/tiny-imagenet', split='valid')
test_loader = torch.utils.data.DataLoader(testset, batch_size=bs, shuffle=False, num_workers=0, collate_fn=collate_fn_test)

#dataset = datasets.CIFAR10(root='./data',
#                           train=False,
#                           transform=transform,
#                           download=True)

# test_loader = DataLoader(dataset,
#                                   batch_size=batch_size,
#                                   shuffle=False,
#                                   num_workers=0)

In [129]:
len(testset['image'])

10000

In [114]:
#sys.path.insert(0, '..')
import robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy


In [130]:
print('[Data loaded]')

acc_mlp_list = []
acc_cnn_list = []

for batch in test_loader:
    images, labels = batch
    acc_mlp = clean_accuracy(model_mlp, images.to(device), labels.to(device))
    acc_mlp_list.append(acc_mlp)
    
    acc_cnn = clean_accuracy(model_resnet, images.to(device), labels.to(device))
    acc_cnn_list.append(acc_cnn)

[Data loaded]


In [131]:
acc_mlp = np.array(acc_mlp_list)
acc = acc_mlp.mean()
print('[MLP loaded]')
print('MLP Acc: %2.2f %%'%(acc*100))

acc_cnn = np.array(acc_cnn_list)
acc = acc_cnn.mean()
print('[CNN loaded]')
print('CNN Acc: %2.2f %%'%(acc*100))

[MLP loaded]
MLP Acc: 72.74 %
[CNN loaded]
CNN Acc: 0.60 %


In [ ]:
def compare_adv_attacks(model1, model2, atk_model1, images, labels, iterations=10, device='cpu'):

    model1.eval()
    model2.eval()

    successful_adv_attacks_model1 = []
    successful_adv_attacks_model2 = []

    for it in range(iterations):
        
        adv_images = atk_model1(images, labels)
        predictions = get_pred(model1, adv_images, device)
        predictions = predictions.cuda()
        
        seccessful_adv_images = adv_images[predictions != labels]
        true_labels_of_succ_adv_images = labels[predictions != labels]
        successful_adv_attacks_model1.append(seccessful_adv_images.shape[0])


        predictions = get_pred(model2, seccessful_adv_images, device)
        predictions = predictions.cuda()
        seccessful_adv_images = seccessful_adv_images[predictions != true_labels_of_succ_adv_images]
        successful_adv_attacks_model2.append(seccessful_adv_images.shape[0])

    return successful_adv_attacks_model1, successful_adv_attacks_model2




In [ ]:
N_exp = 20

atk_mlp = PGD(model_mlp, eps=8/255, alpha=2/225, steps=10, random_start=True)
# atk_mlp.set_normalization_used(mean=mean, std=std)


l1_list_mlp = []
l2_list_mlp = []
# 100 batches
for i,batch in enumerate(test_loader):
    print("batch:", i)
    ll1_list = []
    ll2_list = []
    # N_exp times
    for i in range(5):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        
        l1, l2 = compare_adv_attacks(model_mlp, model_resnet, atk_mlp, images, labels, 1, device)
        ll1_list.append(l1)
        ll2_list.append(l2)
    
    l1_list_mlp.append(ll1_list)
    l2_list_mlp.append(ll2_list)

In [ ]:
len(l1_list)

In [ ]:
ll2_list

In [ ]:
mlp_adv.shape

In [ ]:
np.squeeze(mlp_adv,2).shape

In [ ]:
cnn_adv.shape

In [ ]:
# Trained on mlp Tested on resnet

In [ ]:
mlp_adv = np.array(l1_list_mlp)
mlp_adv = np.squeeze(mlp_adv,2)
mlp_adv = mlp_adv.sum(axis=0)

mlp_adv_mean = mlp_adv.mean(axis=0)
mlp_adv_std = mlp_adv.std(axis=0)
print('mlp_adv_mean: ', mlp_adv_mean)
print('mlp_adv_std: ', mlp_adv_std)

cnn_adv = np.array(l2_list_mlp) 
cnn_adv = np.squeeze(cnn_adv,2)
cnn_adv = cnn_adv.sum(axis=0)

cnn_adv_mean = cnn_adv.mean(axis=0)
cnn_adv_std = cnn_adv.std(axis=0)
print('cnn_adv_mean: ', cnn_adv_mean)
print('cnn_adv_std: ', cnn_adv_std)



In [ ]:
print("ratio: ", (cnn_adv_mean/mlp_adv_mean)*100)

In [ ]:
cnn_adv

In [ ]:
N_exp = 20

atk_cnn = PGD(model_resnet, eps=8/255, alpha=2/225, steps=10, random_start=True)
# atk_cnn.set_normalization_used(mean=mean, std=std)


l1_list_cnn = []
l2_list_cnn = []
# 100 batches
for i,batch in enumerate(test_loader):
    print("batch:", i)
    ll1_list = []
    ll2_list = []
    # N_exp times
    for i in range(5):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        l1, l2 = compare_adv_attacks(model_resnet, model_mlp, atk_cnn, images, labels, 1, device)
        ll1_list.append(l1)
        ll2_list.append(l2)
    
    l1_list_cnn.append(ll1_list)
    l2_list_cnn.append(ll2_list)

In [ ]:
# Trained on resnet Tested on mlp

In [ ]:
cnn_adv = np.array(l1_list_cnn)
cnn_adv = np.squeeze(cnn_adv,2)
cnn_adv = cnn_adv.sum(axis=0)

cnn_adv_mean = cnn_adv.mean(axis=0)
cnn_adv_std = cnn_adv.std(axis=0)
print('cnn_adv_mean: ', cnn_adv_mean)
print('cnn_adv_std: ', cnn_adv_std)

mlp_adv = np.array(l2_list_cnn) 
mlp_adv = np.squeeze(mlp_adv,2)
mlp_adv = mlp_adv.sum(axis=0)

mlp_adv_mean = mlp_adv.mean(axis=0)
mlp_adv_std = mlp_adv.std(axis=0)
print('mlp_adv_mean: ', mlp_adv_mean)
print('mlp_adv_std: ', mlp_adv_std)

In [ ]:
print("ratio: ", (mlp_adv_mean/cnn_adv_mean)*100)

In [ ]:
# PGD: Inside normalization

In [ ]:
mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)

transform = transforms.Compose([
    #transforms.Resize((crop_resolution, crop_resolution)),
    transforms.ToTensor(),
    # transforms.Normalize(mean, std),
])

batch_size = 100

dataset = datasets.CIFAR10(root='./data',
                           train=False,
                           transform=transform,
                           download=True)

test_loader = DataLoader(dataset,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  num_workers=0)

In [ ]:
N_exp = 20

atk_mlp = PGD(model_mlp, eps=8/255, alpha=2/225, steps=10, random_start=True)
atk_mlp.set_normalization_used(mean=mean, std=std)


l1_list_mlp = []
l2_list_mlp = []
# 100 batches
for i,batch in enumerate(test_loader):
    print("batch:", i)
    ll1_list = []
    ll2_list = []
    # N_exp times
    for i in range(5):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        
        l1, l2 = compare_adv_attacks(model_mlp, model_resnet, atk_mlp, images, labels, 1, device)
        ll1_list.append(l1)
        ll2_list.append(l2)
    
    l1_list_mlp.append(ll1_list)
    l2_list_mlp.append(ll2_list)



In [ ]:
# Trained on cnn Tested on mlp

In [ ]:
mlp_adv = np.array(l1_list_mlp)
mlp_adv = np.squeeze(mlp_adv,2)
mlp_adv = mlp_adv.sum(axis=0)

mlp_adv_mean = mlp_adv.mean(axis=0)
mlp_adv_std = mlp_adv.std(axis=0)
print('mlp_adv_mean: ', mlp_adv_mean)
print('mlp_adv_std: ', mlp_adv_std)

cnn_adv = np.array(l2_list_mlp) 
cnn_adv = np.squeeze(cnn_adv,2)
cnn_adv = cnn_adv.sum(axis=0)

cnn_adv_mean = cnn_adv.mean(axis=0)
cnn_adv_std = cnn_adv.std(axis=0)
print('cnn_adv_mean: ', cnn_adv_mean)
print('cnn_adv_std: ', cnn_adv_std)

In [ ]:
print("ratio: ", (cnn_adv_mean/mlp_adv_mean)*100)

In [ ]:
N_exp = 20

atk_cnn = PGD(model_resnet, eps=8/255, alpha=2/225, steps=10, random_start=True)
atk_cnn.set_normalization_used(mean=mean, std=std)


l1_list_cnn = []
l2_list_cnn = []
# 100 batches
for i,batch in enumerate(test_loader):
    print("batch:", i)
    ll1_list = []
    ll2_list = []
    # N_exp times
    for i in range(5):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        l1, l2 = compare_adv_attacks(model_resnet, model_mlp, atk_cnn, images, labels, 1, device)
        ll1_list.append(l1)
        ll2_list.append(l2)
    
    l1_list_cnn.append(ll1_list)
    l2_list_cnn.append(ll2_list)

In [ ]:
# Trained on cnn Tested on mlp

In [ ]:
cnn_adv = np.array(l1_list_cnn)
cnn_adv = np.squeeze(cnn_adv,2)
cnn_adv = cnn_adv.sum(axis=0)

cnn_adv_mean = cnn_adv.mean(axis=0)
cnn_adv_std = cnn_adv.std(axis=0)
print('cnn_adv_mean: ', cnn_adv_mean)
print('cnn_adv_std: ', cnn_adv_std)

mlp_adv = np.array(l2_list_cnn) 
mlp_adv = np.squeeze(mlp_adv,2)
mlp_adv = mlp_adv.sum(axis=0)

mlp_adv_mean = mlp_adv.mean(axis=0)
mlp_adv_std = mlp_adv.std(axis=0)
print('mlp_adv_mean: ', mlp_adv_mean)
print('mlp_adv_std: ', mlp_adv_std)

In [ ]:
print("ratio: ", (mlp_adv_mean/cnn_adv_mean)*100)

In [ ]:
model1 = model_mlp
model2 = model_resnet
atk_model1 = atk
iterations = 1

In [ ]:
model1.eval()
model2.eval()

successful_adv_attacks_model1 = []
successful_adv_attacks_model2 = []

for it in range(iterations):
    print("Iteration:", it)
    adv_images = atk_model1(images, labels)
    predictions = get_pred(model1, adv_images, device)

    seccessful_adv_images = adv_images[predictions.cuda() != labels]
    true_labels_of_succ_adv_images = labels[predictions != labels]
    successful_adv_attacks_model1.append(seccessful_adv_images.shape[0])

    predictions = get_pred(model2, seccessful_adv_images, device)
    seccessful_adv_images = seccessful_adv_images[predictions != true_labels_of_succ_adv_images]
    successful_adv_attacks_model2.append(seccessful_adv_images.shape[0])

In [ ]:
predictions

In [ ]:
labels

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Assuming a single-GPU setup
    device_properties = torch.cuda.get_device_properties(device)
    
    total_memory = device_properties.total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    cached_memory = torch.cuda.memory_reserved(device)

    print(f"Total GPU Memory: {total_memory / 1e9} GB")
    print(f"Allocated Memory: {allocated_memory / 1e9} GB")
    print(f"Cached Memory: {cached_memory / 1e9} GB")
else:
    print("CUDA is not available.")

In [ ]:
l1_list = []
l2_list = []

In [ ]:

    
atk = PGD(model_mlp, eps=8/255, alpha=2/225, steps=10, random_start=True)
atk.set_normalization_used(mean=mean, std=std)
l1, l2 = compare_adv_attcks(model_mlp, model_resnet, atk, images, labels, N_exp, device)


In [ ]:
atk = PGD(model_resnet, eps=8/255, alpha=2/225, steps=10, random_start=True)
atk.set_normalization_used(mean=mean, std=std)
l11, l22 = compare_adv_attcks(model_resnet, model_mlp, atk, images, labels, 2)

In [ ]:
print(l1, l2)

In [ ]:
print(l11, l22)

In [ ]:
idx = 0
pre = get_pred(model_resnet, adv_images[idx:idx+1], device)
imshow(adv_images[idx:idx+1], title="True:%d, Pre:%d"%(labels[idx], pre))

In [ ]:
idx = 0
pre = get_pred(model, adv_images1[idx:idx+1].flatten(1), device)
imshow(adv_images1[idx:idx+1], title="True:%d, Pre:%d"%(labels[idx], pre))